In [241]:
import pandas as pd
import pickle
import os

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [240]:
# Cargar los datos
df_entrenamiento = pd.read_csv('train.csv')
df_prueba = pd.read_csv('test.csv')

print("Dimensiones de los datos de entrenamiento:", df_entrenamiento.shape)
print("Dimensiones de los datos de prueba:", df_prueba.shape)

Dimensiones de los datos de entrenamiento: (9239, 10)
Dimensiones de los datos de prueba: (2309, 9)


In [11]:
# Convertir participación a valor numérico
df_entrenamiento['engagement'] = df_entrenamiento['engagement'].astype(int)

# Separar características y etiquetas
X = df_entrenamiento.drop(columns=['id', 'engagement'])
y = df_entrenamiento['engagement']

# Normalizar las características
escalador = StandardScaler()
X_escalada = escalador.fit_transform(X)

# Dividir los datos en conjunto de entrenamiento y de validación
X_entrenamiento, X_val,y_entrenamiento, y_val \
    = train_test_split(X_escalada, y, test_size=0.2, random_state=42)

print("Dimensiones de los datos con los que se entrenará:",X_entrenamiento.shape)
print("Dimensiones de los datos de validación:", X_val.shape)

Dimensiones de los datos con los que se entrenará: (7391, 8)
Dimensiones de los datos de validación: (1848, 8)


In [243]:
# Definir los modelos
modelos = {
    'Regresión logística': LogisticRegression(max_iter=1000, random_state=42),
    'k vecinos más cercanos': KNeighborsClassifier(),
    'Clasificador de margen máximo': SVC(probability=True, random_state=42),
    'Árbol de decisión': DecisionTreeClassifier(),
    'Bosque aleatorio': RandomForestClassifier(random_state=42),
    'Potencialización de gradiente': GradientBoostingClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42),
    'LightGBM': LGBMClassifier(random_state=42)
}

In [245]:
# Definir la cuadrícula de hiperparámetros
cuadriculas_params = {
    'Regresión logística': [
        {
            'C': [0.01, 0.02, 0.05, 0.1, 1],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear', 'saga'],
            'class_weight': [None, 'balanced']
        },
        {
            'C': [0.01, 0.02, 0.05, 0.1, 1],
            'penalty': ['elasticnet'],
            'solver': ['saga'],
            'class_weight': [None, 'balanced'],
            'l1_ratio': [0.1, 0.5, 0.9]
        }
    ],
    'k vecinos más cercanos': {
        'n_neighbors': [53, 55],
        'weights': ['uniform', 'distance'],
        'p': [2, 3],
    },
    'Clasificador de margen máximo': [
        {
            'C': [0.001, 0.01],
            'kernel': ['rbf'],
            'gamma': ['auto', 0.01]
        },
        {
            'C': [0.001, 0.01],
            'kernel': ['sigmoid'],
            'gamma': ['auto', 0.001],
            'coef0': [0.1, 1.0]
        },
        {
            'C': [2, 3],
            'kernel': ['poly'],
            'gamma': [0.04, 0.05],
            'degree': [2, 3],
            'coef0': [13.0, 19.0]
        }
    ],
    'Árbol de decisión': {
        'max_depth': [5, None],
        'min_samples_split': [5, 6],
        'min_samples_leaf': [4, 5],
        'max_features': [0.5, 0.8],
        'criterion': ['entropy', 'log_loss'],
        'splitter': ['best', 'random'],
        'class_weight': ['balanced', None],
        'max_leaf_nodes': [30, 40, None],
        'min_weight_fraction_leaf': [0.0, 0.01],
        'min_impurity_decrease': [0.0, 0.01]
    },
    'Bosque aleatorio': {
        'n_estimators': [100, 200],
        'max_depth': [10, 15],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'max_features': ['log2', 0.75],
        'class_weight': ['balanced', None],
        'criterion': ['gini', 'entropy']
    },
    'Potencialización de gradiente': {
        'n_estimators': [100, 200],
        'max_depth': [5, 10],
        'learning_rate': [0.05, 0.1],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'loss': ['log_loss', 'exponential'],
        'min_weight_fraction_leaf': [0.02, 0.03]
    },
    'XGBoost': {
        'n_estimators': [97, 100],
        'max_depth': [5, 10],
        'learning_rate': [0.05,0.1],
        'min_child_weight': [2, 5],
        'subsample': [0.9, 1],
        'gamma': [0.1, 0.2],
        'reg_lambda': [0, 1],
        'objective': ['binary:logistic', 'binary:logitraw'],
        'monotone_constraints': [(1, 0), (1, -1)],
        'reg_alpha': [0, 1],
        'scale_pos_weight': [1, 2],
        'booster': ['gbtree', 'dart']
    },
    'LightGBM': {
        'n_estimators': [300, 350],
        'max_depth': [-1, 10],
        'learning_rate': [0.01, 0.02],
        'num_leaves': [30, 40],
        'min_child_samples': [20, 30]
    }
}

In [246]:
# Evaluar los modelos
mejor_puntaje = -float('inf')
mejor_modelo = None
resultados = {}

for nombre, modelo in modelos.items():
    print("Modelo:", nombre)

    # Revisar si el modelo ya existe
    if os.path.exists(f'{nombre}.pkl'):
        with open(f'{nombre}.pkl', 'rb') as archivo:
            datos = pickle.load(archivo)
            modelo = datos['modelo']
            parametros = datos['parametros']
            puntaje = datos['puntaje']
            print("Modelo cargado exitosamente.")
    else:
        # Validación cruzada con el criterio de área bajo la curva característica operativa del receptor (ROC AUC)
        busqueda = GridSearchCV(estimator=modelo, param_grid=cuadriculas_params[nombre], cv=5, scoring='roc_auc', n_jobs=-1, verbose=3)

        # Entrenar el modelo
        busqueda.fit(X_entrenamiento, y_entrenamiento)
        modelo = busqueda.best_estimator_

        # Obtener los mejores hiperparámetros
        parametros = busqueda.best_params_

        # Evaluar el modelo en el conjunto de validación
        y_val_pred = busqueda.predict_proba(X_val)[:, 1]
        puntaje = roc_auc_score(y_val, y_val_pred)

        with open(f'{nombre}.pkl', 'wb') as archivo:
            pickle.dump({'modelo': mejor_modelo,
                        'parametros': parametros,
                        'puntaje': mejor_puntaje}, archivo)
    
        print(f"Información guardada en {nombre}.pkl")

    resultados[nombre] = {
        'mejores_params': parametros,
        'roc_auc': puntaje
    }

    print(f"Mejores hiperparámetros: {parametros}, ROC AUC (validación) = {puntaje:.4f}")

    # Guardar el mejor modelo
    if puntaje > mejor_puntaje:
        mejor_puntaje = puntaje
        mejor_modelo = busqueda.best_estimator_

print(f"\nModelo con el mejor rendimiento: {mejor_modelo}")

Modelo: Regresión logística
Fitting 5 folds for each of 70 candidates, totalling 350 fits
Información guardada en Regresión logística.pkl
Mejores hiperparámetros: {'C': 0.02, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}, ROC AUC (validación) = 0.8784
Modelo: k vecinos más cercanos
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Información guardada en k vecinos más cercanos.pkl
Mejores hiperparámetros: {'n_neighbors': 55, 'p': 2, 'weights': 'distance'}, ROC AUC (validación) = 0.8768
Modelo: Clasificador de margen máximo
Fitting 5 folds for each of 28 candidates, totalling 140 fits
Información guardada en Clasificador de margen máximo.pkl
Mejores hiperparámetros: {'C': 3, 'coef0': 19.0, 'degree': 3, 'gamma': 0.04, 'kernel': 'poly'}, ROC AUC (validación) = 0.8907
Modelo: Árbol de decisión
Fitting 5 folds for each of 1536 candidates, totalling 7680 fits
Información guardada en Árbol de decisión.pkl
Mejores hiperparámetros: {'class_weight': None, 'criteri

c:\Users\seelro06\Downloads\uanl-payroll-analysis\uanl-payroll-analysis\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Información guardada en Bosque aleatorio.pkl
Mejores hiperparámetros: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 10, 'max_features': 0.75, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}, ROC AUC (validación) = 0.8925
Modelo: Potencialización de gradiente
Fitting 5 folds for each of 128 candidates, totalling 640 fits


c:\Users\seelro06\Downloads\uanl-payroll-analysis\uanl-payroll-analysis\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Información guardada en Potencialización de gradiente.pkl
Mejores hiperparámetros: {'learning_rate': 0.1, 'loss': 'exponential', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.03, 'n_estimators': 100}, ROC AUC (validación) = 0.9045
Modelo: XGBoost
Fitting 5 folds for each of 4096 candidates, totalling 20480 fits


c:\Users\seelro06\Downloads\uanl-payroll-analysis\uanl-payroll-analysis\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Información guardada en XGBoost.pkl
Mejores hiperparámetros: {'booster': 'dart', 'gamma': 0.1, 'learning_rate': 0.05, 'max_depth': 5, 'min_child_weight': 2, 'monotone_constraints': (1, 0), 'n_estimators': 97, 'objective': 'binary:logistic', 'reg_alpha': 1, 'reg_lambda': 0, 'scale_pos_weight': 2, 'subsample': 0.9}, ROC AUC (validación) = 0.8995
Modelo: LightGBM
Fitting 5 folds for each of 32 candidates, totalling 160 fits


c:\Users\seelro06\Downloads\uanl-payroll-analysis\uanl-payroll-analysis\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


[LightGBM] [Info] Number of positive: 732, number of negative: 6659
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000694 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1800
[LightGBM] [Info] Number of data points in the train set: 7391, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.099039 -> initscore=-2.207944
[LightGBM] [Info] Start training from score -2.207944
Información guardada en LightGBM.pkl
Mejores hiperparámetros: {'learning_rate': 0.01, 'max_depth': -1, 'min_child_samples': 30, 'n_estimators': 350, 'num_leaves': 40}, ROC AUC (validación) = 0.9031

Modelo con el mejor rendimiento: GradientBoostingClassifier(loss='exponential', max_depth=5,
                           min_weight_fraction_leaf=0.03, random_state=42)


In [247]:
# Normalizar el conjunto de prueba
X_prueba_escalada = escalador.transform(df_prueba.drop(columns=['id']))

# Predecir con el mejor modelo encontrado
mejor_modelo.fit(X_entrenamiento, y_entrenamiento)
y_prueba_pred_proba = busqueda.predict_proba(X_prueba_escalada)[:, 1]

# Guardar los resultados en un archivo
resultados = pd.DataFrame({'id': df_prueba['id'], 'engagement': y_prueba_pred_proba})
resultados.to_csv('resultados.csv', index=False)